<a href="https://colab.research.google.com/github/ShinwooChoi/ESAA-OB/blob/main/9_22_ESAA_OB_%ED%95%84%EC%82%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

범위:핸즈온 9장 비지도 학습 p.328-347

##9.2 가우시안 혼합
정의: 샘플이 파라미터가 알려지지 않은 여러 개의 혼합된 가우시안 분포에서 생성되었다고 가정하는 확률 모델

특징: 클러스터가 타원형 형태까지 표현 가능

방식: 각 샘플이 여러 클러스터에 속할 확률을 가짐 → 소프트 클러스터링

학습 파라미터: 평균(μ), 공분산(Σ), 혼합 계수(π)

장점: K-평균보다 유연한 클러스터링 가능


###9.2.1 가우시안 혼합을 사용한 이상치 탐지

이상치(outlier): 보통과 다른 샘플

정상치(inlier): 보통 샘플

밀도가 낮은 영역의 샘플을 이상치로 판별

임계값(밀도 기준)을 정해 이상치 비율 조절

###9.2.2 클러스터 개수 선택

k-평균처럼 실루엣 점수 사용 불가

정보 기준 사용

BIC: 모델 단순성(파라미터 적음) 선호

AIC: 복잡한 모델(파라미터 많음) 선호

둘 다 가능도(likelihood)를 기반으로 계산

###9.2.3 베이즈 가우시안 혼합

클러스터 개수를 스스로 조절

BayesianGaussianMixture 클래스 사용

불필요한 클러스터 가중치를 0으로 수렴

클러스터 개수 불확실할 때 유용

###9.2.4 이상치 탐지와 특이치 탐지를 위한 다른 알고리즘

PCA 기반 방법: 재구성 오차를 이용해 이상치 판별

Fast-MCD: 데이터 분포(타원형) 가정 → 멀리 떨어진 샘플을 이상치로 탐지

아이솔레이션 포레스트: 랜덤 트리로 샘플 분리 → 분리 빨리 되는 샘플이 이상치

LOF (Local Outlier Factor): 국소 밀도 비교 → 주변보다 밀도 낮으면 이상치

One-class SVM: 정상 데이터 경계를 학습 → 경계 밖 샘플은 이상치

In [1]:
import numpy as np
from sklearn.datasets import make_moons

x, y = make_moons(n_samples=1000,noise=0.05)

In [2]:
# 데이터 불러오기
from sklearn.datasets import make_blobs
import numpy as np

# make_blobs()의 정확한 인수는 중요하지 않습니다.

blob_centers = np.array([[ 0.2, 2.3], [-1.5 , 2.3], [-2.8, 1.8], [-2.8, 2.8], [-2.8, 1.3]])

blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])

x, y = make_blobs(n_samples=2000, centers=blob_centers, cluster_std=blob_std, random_state=7)


In [3]:
from sklearn.mixture import GaussianMixture
gm=GaussianMixture(n_components=3, n_init=10)
gm.fit(x)

GaussianMixture(n_components=3, n_init=10)

In [4]:
gm.weights_

array([0.59989596, 0.20032189, 0.19978216])

In [5]:
gm.means_

array([[-2.80278963,  1.96665812],
       [ 0.19937187,  2.25813691],
       [-1.48543693,  2.28573643]])

In [6]:
gm.covariances_

array([[[ 9.95376970e-03,  7.16529541e-04],
        [ 7.16529541e-04,  3.96947649e-01]],

       [[ 1.46166449e-01, -4.84946828e-03],
        [-4.84946828e-03,  1.56280672e-01]],

       [[ 8.79984367e-02,  1.46541758e-04],
        [ 1.46541758e-04,  9.01630979e-02]]])

In [7]:
gm.converged_

True

In [8]:
gm.n_iter_

3

In [9]:
gm.predict(x)

array([0, 0, 1, ..., 0, 1, 0])

In [10]:
gm.predict_proba(x)

array([[9.99998809e-001, 5.11211601e-015, 1.19140197e-006],
       [9.99998437e-001, 6.43557803e-016, 1.56262666e-006],
       [4.58103106e-307, 1.00000000e+000, 7.51207529e-016],
       ...,
       [9.99991556e-001, 7.53349575e-015, 8.44387449e-006],
       [6.39866293e-202, 9.99999925e-001, 7.54961300e-008],
       [9.99733221e-001, 3.59569371e-013, 2.66779387e-004]])

In [11]:
x_new,y_new=gm.sample(6)
x_new

array([[-2.899771  ,  1.40674329],
       [-2.79235009,  0.16410221],
       [-2.88870495,  0.90754915],
       [-2.79188553,  2.40794137],
       [-2.91984129,  2.33861103],
       [-1.33688867,  2.07894893]])

In [12]:
y_new

array([0, 0, 0, 0, 0, 2])

In [13]:
gm.score_samples(x)

array([-0.62897413,  0.13004267, -7.15363365, ..., -0.3150245 ,
       -1.62809493, -0.94509528])

In [17]:
densities=gm.score_samples(x)
density_threshold=np.percentile(densities,4)
anomalies=x[densities< density_threshold]

In [20]:
gm.bic(x)

np.float64(5168.669155929095)

In [21]:
gm.aic(x)

np.float64(5073.45381411688)

In [22]:
from sklearn.mixture import BayesianGaussianMixture
bgm=BayesianGaussianMixture(n_components=10, n_init=10)
bgm.fit(x)
np.round(bgm.weights_,2)

array([0.2, 0. , 0.2, 0.2, 0.2, 0.2, 0. , 0. , 0. , 0. ])